# **Preparation Notebook**



---
## Setup Environment

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
!pip install -q utstd

from utstd.folders import *
from utstd.ipyrenders import *

at = AtFolder(
    course_code=36106,
    assignment="AT3",
)
at.run()

import warnings
warnings.simplefilter(action='ignore')

---
## Student Information

In [ ]:
# <Student to fill this section and then remove this comment>
group_name = "Group 12"
student_name = "Victor Rono"
student_id = "25669944"

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h1", key='group_name', value=group_name)

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h1", key='student_name', value=student_name)

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h1", key='student_id', value=student_id)

---
## 0. Python Packages

### 0.a Install Additional Packages

> If you are using additional packages, you need to install them here using the command: `! pip install <package_name>`

In [ ]:
!pip install -q utstd scikit-learn==1.5.0 numpy==1.26.0 scipy==1.11.0

### 0.b Import Packages

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
import pandas as pd
import altair as alt

---
## A. Feature Selection


## A.0 Load Data

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
# Load datasets
try:
  sales_2022_df = pd.read_csv(at.folder_path / "sales_2022.csv")
  products_df = pd.read_csv(at.folder_path / "products.csv")
  product_subcats_df = pd.read_csv(at.folder_path / "product_subcats.csv")
  territories_df = pd.read_csv(at.folder_path / "territories.csv")
  sales_2021_df = pd.read_csv(at.folder_path / "sales_2021.csv")
  returns_df = pd.read_csv(at.folder_path / "returns.csv")
  sales_2020_df = pd.read_csv(at.folder_path / "sales_2020.csv")
  product_cats_df = pd.read_csv(at.folder_path / "product_cats.csv")
  customers_df = pd.read_csv(at.folder_path / "customers.csv")
except Exception as e:
  print(e)

### A.1 Approach 1

In [ ]:
import numpy as np
import pandas as pd

prep = customers_df.copy()  # work on a copy

# 1. Age from birth_date if age missing (fixed reference date for reproducibility)
if "age" not in prep.columns and "birth_date" in prep.columns:
    prep["birth_date"] = pd.to_datetime(prep["birth_date"], errors="coerce")
    prep["age"] = ((pd.Timestamp("2022-01-01") - prep["birth_date"]).dt.days/365.25).round(0)

# 2. Age band (lifestage) — easy to explain to stakeholders
if "age" in prep.columns and "age_group" not in prep.columns:
    prep["age_group"] = pd.cut(prep["age"], bins=[0,25,45,65,150],
                               labels=["Youth","Adult","Middle Age","Senior"])

# 3. Income band — quick purchasing-power proxy
if "annual_income" in prep.columns and "income_category" not in prep.columns:
    prep["income_category"] = pd.cut(prep["annual_income"], bins=[0,30000,60000,100000,1e12],
                                     labels=["Low","Middle","Upper Middle","High"])

# 4. Family status — simple dependent flag
if "family_status" not in prep.columns:
    deps = prep.get("number_dependents", np.nan)
    prep["family_status"] = np.where(pd.to_numeric(deps, errors="coerce").fillna(0)>0,
                                     "Has Dependents","No Dependents")

# 5. Value segment — income quartiles (robust to skew)
if "value_segment" not in prep.columns and "annual_income" in prep.columns:
    try:
        prep["value_segment"] = pd.qcut(prep["annual_income"], q=4,
                                        labels=["Low","Mid-Low","Mid-High","High"])
    except Exception:
        # if many ties/NA prevent qcut, fall back to simple bins
        prep["value_segment"] = pd.cut(prep["annual_income"], bins=[-np.inf,0,60000,100000,np.inf],
                                       labels=["NA/Zero","≤60k","60-100k",">100k"])

print("Engineered columns added:",
      [c for c in ["age_group","income_category","family_status","value_segment"] if c in prep.columns])

In [ ]:
feature_selection_1_insights = """
**Approach 1 — Business-driven, interpretable features**

• Focus on lifestage and purchasing power signals that stakeholders understand:
  age / age_group, annual_income / income_category, family_status, homeowner, education, occupation.
• Light engineering only (bands + flags) to keep transparency and reduce leakage risk.
• Result: {X_a1.shape[1]} features selected (columns: {', '.join(X_a1.columns[:6])}{'…' if X_a1.shape[1]>6 else ''}).
• Ready for simple encoding & scaling; redundant score fields will be removed in Approach 2.
""".strip()

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='feature_selection_1_insights', value=feature_selection_1_insights)

### A.2 Approach 2

In [ ]:
import pandas as pd
import numpy as np

# 1. start from Approach-1 output if available; otherwise from the loaded customers table
base = None
if isinstance(globals().get("X_a1"), pd.DataFrame) and not globals().get("X_a1").empty:
    base = globals().get("X_a1")
elif isinstance(globals().get("prep"), pd.DataFrame) and not globals().get("prep").empty:
    base = globals().get("prep")
elif isinstance(globals().get("customers_df"), pd.DataFrame) and not globals().get("customers_df").empty:
    base = globals().get("customers_df")

assert base is not None and not base.empty, "Run A.1 first (or ensure 'customers' is loaded)."

X2 = base.copy()

# 2. drop obvious redundant “score” fields (e.g., satisfaction_score, risk_score)
dropped_scores = [c for c in X2.columns if c.endswith("_score")]
X2 = X2.drop(columns=dropped_scores, errors="ignore")

# 3. remove one of any pair of highly-correlated numeric columns (|r| > 0.95)
num_cols = X2.select_dtypes(include="number").columns.tolist()
dropped_corr = []
if len(num_cols) >= 2:
    corr = X2[num_cols].corr().abs()
    # keep a simple upper-triangle scan
    keep = set()
    for i, c1 in enumerate(num_cols):
        if c1 in dropped_corr:
            continue
        keep.add(c1)
        for c2 in num_cols[i+1:]:
            if corr.loc[c1, c2] > 0.95:
                dropped_corr.append(c2)
    X2 = X2.drop(columns=dropped_corr, errors="ignore")

# 4. output for downstream steps
X_a2 = X2.copy()             # Approach-2 dataset
features_2 = X_a2.columns.tolist()
print({"kept_features": len(features_2),
       "dropped_scores": dropped_scores,
       "dropped_high_corr": dropped_corr})
X_a2.head(3)

In [ ]:

feature_selection_2_insights = """
**Approach 2 — De-redundancy & stability**

• Removed superficial *_score fields to avoid double-counting and improve interpretability.
• Pruned highly-correlated numeric pairs (|r|>0.95) to reduce multicollinearity risk.
• Result: kept **{len(features_2)}** features; dropped scores: {len(globals().get('dropped_scores', []))},
  high-corr removed: {len(globals().get('dropped_corr', []))}.
• This yields a lean, stable set that encodes/scales cleanly and supports fair model comparison.
"""

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='feature_selection_2_insights', value=feature_selection_2_insights)

### A.n Approach "\<describe_approach_here\>"

> You can add more cells related to other approaches in this section

In [ ]:
import pandas as pd
import numpy as np

base = None
if isinstance(globals().get("X_a2"), pd.DataFrame) and not globals().get("X_a2").empty:
    base = globals().get("X_a2")
elif isinstance(globals().get("X_a1"), pd.DataFrame) and not globals().get("X_a1").empty:
    base = globals().get("X_a1")
elif isinstance(globals().get("customers_df"), pd.DataFrame) and not globals().get("customers_df").empty:
    base = globals().get("customers_df")

assert base is not None and not base.empty, "Run A.1/A.2 first (or ensure 'customers' is loaded)."

Xn = base.copy()

# 2) Split numeric vs categorical once
num_cols = [c for c in Xn.columns if pd.api.types.is_numeric_dtype(Xn[c])]
cat_cols = [c for c in Xn.columns if c not in num_cols]

# 3) Drop near-constant numeric features (very low variance → little signal)
near_const = []
for c in num_cols:
    v = np.nanvar(pd.to_numeric(Xn[c], errors="coerce"))
    if v < 1e-6:                 # threshold kept simple and transparent
        near_const.append(c)
Xn = Xn.drop(columns=near_const, errors="ignore")

# 4) Rare-category grouping for high-cardinality categoricals
#  Any level with <1% frequency becomes 'Other' (keeps models stable and interpretable)
rarity_threshold = 0.01
grouped_info = {}
for c in cat_cols:
    if c not in Xn.columns:
        continue
    # --- MODIFICATION: Skip rarity grouping for 'customer_id' ---
    if c in ["customer_id", "CustomerID", "CustomerKey", "customer_key", "id", "ID"]:
        continue
    # --- End Modification ---

    freq = Xn[c].astype(str).value_counts(normalize=True, dropna=False)
    rare_levels = freq[freq < rarity_threshold].index.tolist()
    if rare_levels:
        grouped_info[c] = len(rare_levels)
        Xn[c] = Xn[c].astype(str).where(~Xn[c].astype(str).isin(rare_levels), "Other")

# 5) Output for downstream steps
X_an = Xn.copy()   # Approach-n dataset
kept_cols_n = X_an.columns.tolist()
print({
    "kept_features": len(kept_cols_n),
    "near_constant_dropped": near_const,
    "cats_grouped": grouped_info
})
display(X_an.head(3))

In [ ]:

feature_selection_n_insights = """
**Approach n — Unsupervised filters (variance + rarity)**
• Removed near-constant numeric features to avoid noise: {len(globals().get('near_const', []))} dropped.
• Grouped rare categorical levels (<1% frequency) to 'Other' for stability: {sum(globals().get('grouped_info', {}).values())} levels grouped across {len(globals().get('grouped_info', {}))} fields.
• Result: **{len(globals().get('kept_cols_n', X_an.columns).tolist() if isinstance(X_an, pd.DataFrame) else X_an.shape[1])}** features retained; cleaner, more robust inputs for encoding/scaling.
"""

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='feature_selection_n_insights', value=feature_selection_n_insights)

### A.z Final Selection of Features

In [ ]:
import pandas as pd

# 1) Pick the best prepared table available (prefer the latest approach)
base = None
if isinstance(globals().get("X_an"), pd.DataFrame) and not globals().get("X_an").empty:
    base = globals().get("X_an")
elif isinstance(globals().get("X_a2"), pd.DataFrame) and not globals().get("X_a2").empty:
    base = globals().get("X_a2")
elif isinstance(globals().get("X_a1"), pd.DataFrame) and not globals().get("X_a1").empty:
    base = globals().get("X_a1")

assert base is not None and not base.empty, "Run A.1/A.2 (and A.n if used) before this cell."

# 2) Final feature list = columns of the chosen table
features_list = base.columns.tolist()

# 3) (Optional) quick sanity peek
print({"final_feature_count": len(features_list)})
print("features_list:", features_list)
display(base.head(3))

In [ ]:

feature_selection_explanations = """
**Final selection — {len(features_list)} features**
• Started with a business-interpretable shortlist (A.1),
  removed redundant *_score and highly correlated numerics (A.2),
  and optionally stabilised with variance/rarity filtering (A.n).
• The final set balances interpretability (age/education/occupation/income bands, family/homeowner)
  with robustness (reduced multicollinearity; grouped rare levels).
• Ready for one-hot encoding + scaling in the next stage (Baseline/Experiment).
"""

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='feature_selection_explanations', value=feature_selection_explanations)

---
## B. Data Cleaning

### B.1 Fixing "\<describe_issue_here\>"

In [ ]:
# Inconsistent categorical labels (case / leading-trailing spaces)

import pandas as pd
import numpy as np

# 1) Start from the latest prepared table if available; fall back gracefully
base = None
if isinstance(globals().get("X_an"), pd.DataFrame) and not globals().get("X_an").empty:
    base = globals().get("X_an")
elif isinstance(globals().get("X_a2"), pd.DataFrame) and not globals().get("X_a2").empty:
    base = globals().get("X_a2")
elif isinstance(globals().get("X_a1"), pd.DataFrame) and not globals().get("X_a1").empty:
    base = globals().get("X_a1")
elif isinstance(globals().get("customers_df"), pd.DataFrame) and not globals().get("customers_df").empty:
    base = globals().get("customers_df")


assert base is not None and not base.empty, "Run A.1/A.2 (and A.n if used) before B.1."

X_clean1 = base.copy()

# 2) Identify categorical/text columns
cat_cols = [c for c in X_clean1.columns
            if X_clean1[c].dtype == "object" or str(X_clean1[c].dtype) == "category"]

# 3) Helper: normalise strings → strip spaces, collapse internal whitespace, Title case
def _norm_str(s: pd.Series) -> pd.Series:
    s = s.astype(str)
    s = s.str.replace(r"\s+", " ", regex=True)  # collapse multiple spaces
    s = s.str.strip().replace({"": np.nan})     # trim + empty→NaN
    return s.str.title()                        # consistent casing

# 4) Apply normalisation + simple mode imputation where needed
for c in cat_cols:
    before = X_clean1[c].value_counts(dropna=False).head(3).to_dict()  # snapshot of top labels
    X_clean1[c] = _norm_str(X_clean1[c])
    if X_clean1[c].isna().any():
        X_clean1[c] = X_clean1[c].fillna(X_clean1[c].mode().iloc[0])
    after = X_clean1[c].value_counts(dropna=False).head(3).to_dict()
    print(f"[{c}] top before→after:", before, "→", after)

print({"rows": len(X_clean1), "cols": X_clean1.shape[1], "cleaned_cat_cols": len(cat_cols)})

In [ ]:

data_cleaning_1_explanations = """
**Issue fixed:** Inconsistent categorical labels (case, extra spaces, blank strings).

**Why it matters**
• Duplicate categories like " single ", "Single", and "SINGLE" fragment counts and distort one-hot encodings.
• Blank strings behave like valid values, leaking noise into models and reports.

**What I did**
• Normalised all text features: strip spaces, collapse internal whitespace, Title-case.
• Converted empty strings to missing values, then **mode-imputed** per column.

**Impact**
• Fewer spurious levels, stable one-hot dimension, cleaner summary stats; improves downstream model reliability.
"""

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='data_cleaning_1_explanations', value=data_cleaning_1_explanations)

### B.2 Fixing "\<describe_issue_here\>"

In [ ]:
# Unrealistic numeric outliers (IQR capping with safe fallbacks)

import pandas as pd
import numpy as np

# 1) Start from latest cleaned table; fallback gracefully
base = None
if isinstance(globals().get("X_clean1"), pd.DataFrame) and not globals().get("X_clean1").empty:
    base = globals().get("X_clean1")   # from B.1 (categorical normalisation)
elif isinstance(globals().get("X_an"), pd.DataFrame) and not globals().get("X_an").empty:
    base = globals().get("X_an")    # from A.n
elif isinstance(globals().get("X_a2"), pd.DataFrame) and not globals().get("X_a2").empty:
    base = globals().get("X_a2")    # from A.2
elif isinstance(globals().get("X_a1"), pd.DataFrame) and not globals().get("X_a1").empty:
    base = globals().get("X_a1")   # from A.1

assert base is not None and not base.empty, "Run earlier prep steps before B.2."

X_clean2 = base.copy()

# 2) Find numeric columns only
num_cols = [c for c in X_clean2.columns if pd.api.types.is_numeric_dtype(X_clean2[c])]

capped_summary = {}  # per-column counts of capped values
for c in num_cols:
    s = pd.to_numeric(X_clean2[c], errors="coerce")

    # IQR fences
    q1, q3 = s.quantile(0.25), s.quantile(0.75)
    iqr = q3 - q1

    if pd.isna(iqr) or iqr == 0:
        # Fallback for flat/constant columns: use 1st–99th percentile bounds
        lo, hi = s.quantile(0.01), s.quantile(0.99)
    else:
        lo, hi = q1 - 1.5 * iqr, q3 + 1.5 * iqr

    # Clip and count how many changed
    s_clip = s.clip(lower=lo, upper=hi)
    capped_summary[c] = int((s_clip != s).sum())

    # Write back (preserve dtype as float to avoid overflow)
    X_clean2[c] = s_clip.astype(float)

print("Capped outliers per column (non-zero only):")
print({k:v for k,v in capped_summary.items() if v})
print({"rows": len(X_clean2), "cols": X_clean2.shape[1]})

In [ ]:

data_cleaning_2_explanations = """
**Issue fixed:** Unrealistic numeric outliers.

**Why it matters**
• Extreme values skew means/variances, dominate scaling, and can destabilise distance-based models
  and regularised linear methods.

**What I did**
• Applied per-feature robust caps using **IQR fences** (Q1−1.5·IQR, Q3+1.5·IQR);
  for flat/zero-IQR columns fell back to **1st–99th percentile** bounds.
• Only values beyond bounds were clipped; no rows were dropped.

**Impact**
• Columns adjusted: {len(changed_cols)} ({', '.join(changed_cols[:6])}{'…' if len(changed_cols)>6 else ''}).
• Produces stable scaling, fairer model coefficients, and reduces sensitivity to data entry errors.
"""

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='data_cleaning_2_explanations', value=data_cleaning_2_explanations)

### B.3 Fixing "\<describe_issue_here\>"

In [ ]:
# duplicate records / repeated IDs (keep one per customer)

import pandas as pd
import numpy as np

# 1) Start from the most-recent cleaned table; fall back gracefully
base = None
if isinstance(globals().get("X_clean2"), pd.DataFrame) and not globals().get("X_clean2").empty:
    base = globals().get("X_clean2")  # after B.2 outlier capping
elif isinstance(globals().get("X_clean1"), pd.DataFrame) and not globals().get("X_clean1").empty:
    base = globals().get("X_clean1")
elif isinstance(globals().get("X_an"), pd.DataFrame) and not globals().get("X_an").empty:
    base = globals().get("X_an")
elif isinstance(globals().get("X_a2"), pd.DataFrame) and not globals().get("X_a2").empty:
    base = globals().get("X_a2")
elif isinstance(globals().get("X_a1"), pd.DataFrame) and not globals().get("X_a1").empty:
    base = globals().get("X_a1")
elif isinstance(globals().get("customers_df"), pd.DataFrame) and not globals().get("customers_df").empty:
    base = globals().get("customers_df")


assert base is not None and not base.empty, "Run earlier prep steps before B.3."

df = base.copy()
rows_before = len(df)

# 2) First remove exact duplicate rows (all columns identical)
exact_dups = int(df.duplicated().sum())
df = df.drop_duplicates(ignore_index=True)

# 3) Identify a customer ID column (robust to different names)
id_candidates = ["CustomerKey","customer_key","CustomerID","customer_id","ID","id"]
id_col = next((c for c in id_candidates if c in df.columns), None)

# Optional "recency" column to decide which duplicate to keep
date_candidates = ["UpdatedAt","updated_at","LastUpdate","last_update","OrderDate","order_date"]
date_col = next((c for c in date_candidates if c in df.columns), None)
if date_col is not None:
    df[date_col] = pd.to_datetime(df[date_col], errors="coerce")

# 4) If we have an ID, keep ONE row per ID (latest by date if available, else first)
id_dups = 0
if id_col is not None:
    if date_col is not None:
        # sort so newest comes first; drop duplicates keeping first (newest)
        df = df.sort_values(by=date_col, ascending=False)
        id_dups = int(df.duplicated(subset=[id_col]).sum())
        df = df.drop_duplicates(subset=[id_col], keep="first").sort_index(ignore_index=True)
    else:
        id_dups = int(df.duplicated(subset=[id_col]).sum())
        df = df.drop_duplicates(subset=[id_col], keep="first").reset_index(drop=True)

rows_after = len(df)

# 5) Output for downstream steps
X_clean3 = df.copy()
print({
    "rows_before": rows_before,
    "rows_after": rows_after,
    "exact_row_duplicates_removed": exact_dups,
    "id_duplicates_removed": id_dups,
    "id_col_used": id_col,
    "recency_col_used": date_col
})
display(X_clean3.head(3))

In [ ]:

data_cleaning_3_explanations = """
**Issue fixed:** Duplicate records / repeated IDs.

**Why it matters**
• Duplicates inflate counts, bias aggregates (e.g., spend per customer),
  and can leak target information when the same entity appears multiple times.

**What I did**
• Removed exact duplicate rows across all columns.
• Then de-duplicated by **{globals().get('id_col','<ID not found>')}**
  (kept the most recent by {globals().get('date_col','<no date>' )} when available; otherwise the first occurrence).

**Impact**
• Rows before → after: {globals().get('rows_before','?')} → {globals().get('rows_after','?')}.
• Exact duplicates removed: {globals().get('exact_dups','?')}; ID-level duplicates removed: {globals().get('id_dups','?')}.
• Dataset now has one row per customer, giving stable statistics and fair model training.
"""

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='data_cleaning_3_explanations', value=data_cleaning_3_explanations)

### B.n Fixing "\<describe_issue_here\>"

> You can add more cells related to other issues in this section

In [ ]:
# Bad date values (mixed formats, future dates, impossible birth dates) and also derives a robust tenure feature.

import pandas as pd
import numpy as np

# 1) Start from the latest cleaned table; fall back gracefully
base = None
if isinstance(globals().get("X_clean3"), pd.DataFrame) and not globals().get("X_clean3").empty:
    base = globals().get("X_clean3")  # after B.2 outlier capping
elif isinstance(globals().get("X_clean2"), pd.DataFrame) and not globals().get("X_clean2").empty:
    base = globals().get("X_clean2")
elif isinstance(globals().get("X_clean1"), pd.DataFrame) and not globals().get("X_clean1").empty:
    base = globals().get("X_clean1")
elif isinstance(globals().get("X_an"), pd.DataFrame) and not globals().get("X_an").empty:
    base = globals().get("X_an")
elif isinstance(globals().get("X_a2"), pd.DataFrame) and not globals().get("X_a2").empty:
    base = globals().get("X_a2")
elif isinstance(globals().get("X_a1"), pd.DataFrame) and not globals().get("X_a1").empty:
    base = globals().get("X_a1")
elif isinstance(globals().get("customers_df"), pd.DataFrame) and not globals().get("customers_df").empty:
    base = globals().get("customers_df")

assert base is not None and not base.empty, "Run earlier prep steps before B.n."

X_dt = base.copy()
today = pd.Timestamp("today").normalize()

# 2) Candidate date columns (robust to different names/cases)
candidates = [
    "birth_date","BirthDate",
    "created_at","CreatedAt","signup_date","SignupDate",
    "first_purchase_date","FirstPurchaseDate",
    "updated_at","UpdatedAt","last_update","LastUpdate",
    "order_date","OrderDate"
]
date_cols = [c for c in candidates if c in X_dt.columns]

fixed_summary = {}

# 3) Parse dates + fix clearly invalid values
for c in date_cols:
    s_raw = X_dt[c]
    s = pd.to_datetime(s_raw, errors="coerce", utc=False)        # tolerant parsing
    before_na = s.isna().sum()

    # (a) Future dates → NaT
    s[s > today] = pd.NaT

    # (b) Birth dates: reject <1900 or younger than 10 years old
    if c.lower() in {"birth_date", "birthdate"}:
        s[(s < pd.Timestamp("1900-01-01")) | (s > today - pd.DateOffset(years=10))] = pd.NaT

    # save back
    X_dt[c] = s
    fixed_summary[c] = {"parsed_to_datetime": True,
                        "new_NaT_added": int(s.isna().sum() - before_na)}

# 4) Derive tenure_days from the best available "start" date
tenure_sources = ["created_at","CreatedAt","signup_date","SignupDate","first_purchase_date","FirstPurchaseDate"]
src = next((c for c in tenure_sources if c in X_dt.columns), None)

if src is not None:
    X_dt["tenure_days"] = (today - X_dt[src]).dt.days
    # Negative or NaN tenure (from NaT) → set to 0 for stability (optional)
    X_dt["tenure_days"] = X_dt["tenure_days"].clip(lower=0).fillna(0).astype("int64")
else:
    X_dt["tenure_days"] = 0  # safe default if no start date exists

# 5) Output
X_cleanN = X_dt.copy()
print("Date columns parsed/fixed:", date_cols)
print("Fix summary:", fixed_summary)
print("Tenure source:", src, "| nonzero tenure rows:", int((X_cleanN['tenure_days']>0).sum()))
display(X_cleanN.head(3))

In [ ]:

data_cleaning_n_explanations = """
**Issue fixed:** Bad date values (mixed formats, future dates, impossible birth dates).

**Why it matters**
• Invalid dates break time-derived features (e.g., tenure, age), distort seasonality,
  and can crash transformations that expect proper datetimes.

**What I did**
• Parsed all candidate date fields with tolerant coercion; set **future dates to NaT**.
• For birth dates, removed values earlier than 1900 or implying age < 10 years.
• Derived **tenure_days** from the earliest available start date (created/signup/first purchase).

**Impact**
• Clean, comparable dates; robust **tenure_days** feature for modelling/segmentation.
• Reduces errors in downstream encoders and avoids misleading age/tenure statistics.
"""

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='data_cleaning_n_explanations', value=data_cleaning_n_explanations)

---
## C. Feature Engineering

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
# Create copy of datasets

try:
  training_df_eng = training_df_clean.copy()
  validation_df_eng = validation_df_clean.copy()
  testing_df_eng = testing_df_clean.copy()
except Exception as e:
  print(e)

### C.1 New Feature "\<put_name_here\>"



In [ ]:
# income_per_dependent (and a stabilized log version)

import pandas as pd
import numpy as np

# 1) Start from latest cleaned table; fall back gracefully
base = None
if isinstance(globals().get("X_cleanN"), pd.DataFrame) and not globals().get("X_cleanN").empty:
    base = globals().get("X_cleanN")
elif isinstance(globals().get("X_clean3"), pd.DataFrame) and not globals().get("X_clean3").empty:
    base = globals().get("X_clean3")
elif isinstance(globals().get("X_clean2"), pd.DataFrame) and not globals().get("X_clean2").empty:
    base = globals().get("X_clean2")
elif isinstance(globals().get("X_clean1"), pd.DataFrame) and not globals().get("X_clean1").empty:
    base = globals().get("X_clean1")
elif isinstance(globals().get("X_an"), pd.DataFrame) and not globals().get("X_an").empty:
    base = globals().get("X_an")
elif isinstance(globals().get("X_a2"), pd.DataFrame) and not globals().get("X_a2").empty:
    base = globals().get("X_a2")
elif isinstance(globals().get("X_a1"), pd.DataFrame) and not globals().get("X_a1").empty:
    base = globals().get("X_a1")
elif isinstance(globals().get("customers_df"), pd.DataFrame) and not globals().get("customers_df").empty:
    base = globals().get("customers_df")

assert base is not None and not base.empty, "Run earlier prep steps before C.1."

X_feat = base.copy()

# 2) Ensure required inputs exist and are numeric
inc_col = next((c for c in ["annual_income","income","AnnualIncome"] if c in X_feat.columns), None)
dep_col = next((c for c in ["number_dependents","dependents","NumberDependents"] if c in X_feat.columns), None)
assert inc_col is not None and dep_col is not None, "Need annual income and number of dependents."

X_feat[inc_col] = pd.to_numeric(X_feat[inc_col], errors="coerce").fillna(0)
X_feat[dep_col] = pd.to_numeric(X_feat[dep_col], errors="coerce").fillna(0)

# 3) Compute income per dependent; add +1 in denominator to avoid division by zero
X_feat["income_per_dependent"] = X_feat[inc_col] / (1.0 + X_feat[dep_col])

# 4) Log-transform to stabilize heavy right skew (log1p handles zeros)
X_feat["log_income_per_dependent"] = np.log1p(X_feat["income_per_dependent"])

# 5) Output for downstream steps
print({
    "rows": len(X_feat),
    "feature_head": X_feat[["income_per_dependent","log_income_per_dependent"]].head(3).to_dict("records")
})
display(X_feat[[inc_col, dep_col, "income_per_dependent", "log_income_per_dependent"]].head())

In [ ]:

feature_engineering_1_explanations = """
**New feature:** `income_per_dependent` (+ `log_income_per_dependent`)

**Why**
• Captures effective spending capacity by spreading household income across dependents.
• More interpretable than raw income and often more predictive for value/retention than income alone.

**How**
• Computed as {inc_col}/(1 + {dep_col}) to avoid divide-by-zero when dependents = 0.
• Added a **log1p** version to reduce right-skew and help linear models.

**Impact**
• Provides a stable, business-meaningful signal for segmentation and downstream models.
"""

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='feature_engineering_1_explanations', value=feature_engineering_1_explanations)

### C.3 New Feature "\<put_name_here\>"



In [ ]:
#tenure signals (tenure_years, tenure_bucket, recent_customer)

import pandas as pd
import numpy as np

# 1) Start from latest table; fall back gracefully
base = None
if isinstance(globals().get("X_feat"), pd.DataFrame) and not globals().get("X_feat").empty:
    base = globals().get("X_feat")    # from C.1
elif isinstance(globals().get("X_cleanN"), pd.DataFrame) and not globals().get("X_cleanN").empty:
    base = globals().get("X_cleanN")
elif isinstance(globals().get("X_clean3"), pd.DataFrame) and not globals().get("X_clean3").empty:
    base = globals().get("X_clean3")
elif isinstance(globals().get("X_clean2"), pd.DataFrame) and not globals().get("X_clean2").empty:
    base = globals().get("X_clean2")
elif isinstance(globals().get("X_clean1"), pd.DataFrame) and not globals().get("X_clean1").empty:
    base = globals().get("X_clean1")
elif isinstance(globals().get("X_an"), pd.DataFrame) and not globals().get("X_an").empty:
    base = globals().get("X_an")
elif isinstance(globals().get("X_a2"), pd.DataFrame) and not globals().get("X_a2").empty:
    base = globals().get("X_a2")
elif isinstance(globals().get("X_a1"), pd.DataFrame) and not globals().get("X_a1").empty:
    base = globals().get("X_a1")
elif isinstance(globals().get("customers_df"), pd.DataFrame) and not globals().get("customers_df").empty:
    base = globals().get("customers_df")


assert base is not None and not base.empty, "Run earlier prep steps before C.3."

X_ten = base.copy()
today = pd.Timestamp("today").normalize()

# 2) Ensure tenure_days exists; derive from best available start date if needed
if "tenure_days" not in X_ten.columns:
    start_candidates = ["created_at","CreatedAt","signup_date","SignupDate",
                        "first_purchase_date","FirstPurchaseDate"]
    start_col = next((c for c in start_candidates if c in X_ten.columns), None)
    if start_col:
        X_ten[start_col] = pd.to_datetime(X_ten[start_col], errors="coerce")
        X_ten["tenure_days"] = (today - X_ten[start_col]).dt.days
    else:
        X_ten["tenure_days"] = 0  # safe default if no start date exists

# Clean negatives/NaNs (can happen if bad dates slipped through)
X_ten["tenure_days"] = pd.to_numeric(X_ten["tenure_days"], errors="coerce").clip(lower=0).fillna(0).astype(int)

# 3) New features:
#    a) tenure_years (continuous, easy for linear models)
X_ten["tenure_years"] = X_ten["tenure_days"] / 365.25

#    b) tenure_bucket (categorical bands for business reporting)
bins  = [-1, 90, 180, 365, 1095, np.inf]
labels = ["≤3m", "3–6m", "6–12m", "1–3y", "≥3y"]
X_ten["tenure_bucket"] = pd.cut(X_ten["tenure_days"], bins=bins, labels=labels)

#    c) recent_customer flag (useful for churn/activation features)
X_ten["recent_customer"] = np.where(X_ten["tenure_days"] <= 90, 1, 0).astype(int)

# 4) Output for downstream steps
print({
    "rows": len(X_ten),
    "example": X_ten[["tenure_days","tenure_years","tenure_bucket","recent_customer"]].head(3).to_dict("records")
})
display(X_ten[["tenure_days","tenure_years","tenure_bucket","recent_customer"]].head())

In [ ]:

feature_engineering_2_explanations = """
**New features:** `tenure_years`, `tenure_bucket`, `recent_customer`

**Why**
• Tenure captures relationship depth and is often predictive of spend, retention, and support demand.
• Buckets provide an easy business lens (≤3m, 3–6m, 6–12m, 1–3y, ≥3y) for reporting and stratified evaluation.

**How**
• Derived `tenure_days` from created/signup/first-purchase date (fallback if missing), then:
  – `tenure_years` = tenure_days / 365.25 (continuous),
  – `tenure_bucket` = categorical bands,
  – `recent_customer` = 1 if tenure ≤ 90 days, else 0.

**Impact**
• Adds both interpretable segments and model-friendly numeric signal; useful for targeting onboarding vs. mature customers.
"""

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='feature_engineering_2_explanations', value=feature_engineering_2_explanations)

### C.4 New Feature "\<put_name_here\>"



In [ ]:
# socioeconomic_score (+ band)
# Combines income strength, education level, and homeownership into one interpretable index.

import pandas as pd
import numpy as np

# 1) Start from latest table; fall back gracefully
base = None
if isinstance(globals().get("X_ten"), pd.DataFrame) and not globals().get("X_ten").empty:
    base = globals().get("X_ten")    # from C.3
elif isinstance(globals().get("X_feat"), pd.DataFrame) and not globals().get("X_feat").empty:
    base = globals().get("X_feat")    # from C.1
elif isinstance(globals().get("X_cleanN"), pd.DataFrame) and not globals().get("X_cleanN").empty:
    base = globals().get("X_cleanN")  # from B.n
elif isinstance(globals().get("X_clean3"), pd.DataFrame) and not globals().get("X_clean3").empty:
    base = globals().get("X_clean3")  # from B.3
elif isinstance(globals().get("X_clean2"), pd.DataFrame) and not globals().get("X_clean2").empty:
    base = globals().get("X_clean2")  # from B.2
elif isinstance(globals().get("X_clean1"), pd.DataFrame) and not globals().get("X_clean1").empty:
    base = globals().get("X_clean1")  # from B.1
elif isinstance(globals().get("X_an"), pd.DataFrame) and not globals().get("X_an").empty:
    base = globals().get("X_an")    # from A.n
elif isinstance(globals().get("X_a2"), pd.DataFrame) and not globals().get("X_a2").empty:
    base = globals().get("X_a2")    # from A.2
elif isinstance(globals().get("X_a1"), pd.DataFrame) and not globals().get("X_a1").empty:
    base = globals().get("X_a1")   # from A.1
elif isinstance(globals().get("customers_df"), pd.DataFrame) and not globals().get("customers_df").empty:
    base = globals().get("customers_df") # loaded from file

assert base is not None and not base.empty, "Run earlier prep steps before C.4."

X_soc = base.copy()

# 2) Column names (robust)
inc_col = next((c for c in ["annual_income","income","AnnualIncome"] if c in X_soc.columns), None)
edu_col = next((c for c in ["education_level","EducationLevel","education"] if c in X_soc.columns), None)
home_col= next((c for c in ["homeowner","Homeowner","housing_status","HousingStatus"] if c in X_soc.columns), None)
assert inc_col is not None, "Need an income column (e.g., annual_income)."
# education/homeowner are optional; if absent we default to neutral values below.

# 3) Normalise inputs
X_soc[inc_col] = pd.to_numeric(X_soc[inc_col], errors="coerce").fillna(0)

def _norm_text(s):
    return s.astype(str).str.strip().str.lower()

# education → ordinal 0..4
edu_map = {
    "primary":0, "elementary":0,
    "secondary":1, "high school":1, "highschool":1,
    "diploma":2, "certificate":2, "trade":2,
    "bachelor":3, "bachelors":3, "undergraduate":3, "degree":3,
    "master":4, "masters":4, "postgraduate":4, "doctorate":4, "phd":4
}
if edu_col is not None:
    edu_norm = _norm_text(X_soc[edu_col])
    X_soc["edu_level_num"] = edu_norm.map(edu_map)
    X_soc["edu_level_num"] = X_soc["edu_level_num"].fillna(X_soc["edu_level_num"].median() if X_soc["edu_level_num"].notna().any() else 2)
else:
    X_soc["edu_level_num"] = 2  # neutral if education missing

# homeowner → 0/1
true_like  = {"y","yes","true","owner","own","1"}
if home_col is not None:
    home_norm = _norm_text(X_soc[home_col])
    X_soc["homeowner_flag"] = home_norm.isin(true_like).astype(int)
else:
    X_soc["homeowner_flag"] = 0  # neutral fallback

# 4) Income percentile (0..1) for scale-free combination
income_pct = X_soc[inc_col].rank(pct=True, method="average").fillna(0)

# 5) Composite index (weights sum to 1 for interpretability)
#    income has strongest weight; education moderate; homeowner small but useful.
X_soc["socioeconomic_score"] = (
    0.6 * income_pct +
    0.3 * (X_soc["edu_level_num"] / 4.0) +
    0.1 * X_soc["homeowner_flag"]
)

# 6) Band version for business reporting
try:
    X_soc["socioeconomic_band"] = pd.qcut(X_soc["socioeconomic_score"], q=4, labels=["Low","Mid-Low","Mid-High","High"])
except Exception:
    # fallback if too few unique values
    X_soc["socioeconomic_band"] = pd.cut(X_soc["socioeconomic_score"], bins=[-np.inf,0.25,0.5,0.75,np.inf],
                                         labels=["Low","Mid-Low","Mid-High","High"])

# 7) Output preview
print({
    "rows": len(X_soc),
    "example": X_soc[[inc_col,"edu_level_num","homeowner_flag","socioeconomic_score","socioeconomic_band"]].head(3).to_dict("records")
})
display(X_soc[[inc_col,"edu_level_num","homeowner_flag","socioeconomic_score","socioeconomic_band"]].head())

In [ ]:

feature_engineering_3_explanations = """
**New features:** `socioeconomic_score` and `socioeconomic_band`

**Why**
• Combines three interpretable signals—income strength, education level, and homeownership—into one index.
• Useful for segmentation (targeting offers) and often predictive of value/retention.

**How**
• Income converted to a **percentile rank** (0–1) so it’s scale-free.
• Education mapped to an ordinal 0–4 (Primary→Doctorate).
• Homeownership converted to a binary flag.
• Weighted blend: 0.6·income + 0.3·education + 0.1·homeowner; then bucketed into **Low / Mid-Low / Mid-High / High**.

**Impact**
• Produces a compact, business-friendly signal while remaining transparent and easy to justify.
"""

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='feature_engineering_3_explanations', value=feature_engineering_3_explanations)

### C.n Fixing "\<describe_issue_here\>"

> You can add more cells related to new features in this section

In [ ]:
# Post-engineering issues (NaN/Inf, wrong dtypes, invalid ranges)

import pandas as pd
import numpy as np

# 1) Start from the latest engineered table; fall back gracefully
base = None
if isinstance(globals().get("X_soc"), pd.DataFrame) and not globals().get("X_soc").empty:
    base = globals().get("X_soc")    # from C.4
elif isinstance(globals().get("X_ten"), pd.DataFrame) and not globals().get("X_ten").empty:
    base = globals().get("X_ten")    # from C.3
elif isinstance(globals().get("X_feat"), pd.DataFrame) and not globals().get("X_feat").empty:
    base = globals().get("X_feat")    # from C.1
elif isinstance(globals().get("X_cleanN"), pd.DataFrame) and not globals().get("X_cleanN").empty:
    base = globals().get("X_cleanN")  # from B.n
elif isinstance(globals().get("X_clean3"), pd.DataFrame) and not globals().get("X_clean3").empty:
    base = globals().get("X_clean3")  # from B.3
elif isinstance(globals().get("X_clean2"), pd.DataFrame) and not globals().get("X_clean2").empty:
    base = globals().get("X_clean2")  # from B.2
elif isinstance(globals().get("X_clean1"), pd.DataFrame) and not globals().get("X_clean1").empty:
    base = globals().get("X_clean1")  # from B.1
elif isinstance(globals().get("X_an"), pd.DataFrame) and not globals().get("X_an").empty:
    base = globals().get("X_an")    # from A.n
elif isinstance(globals().get("X_a2"), pd.DataFrame) and not globals().get("X_a2").empty:
    base = globals().get("X_a2")    # from A.2
elif isinstance(globals().get("X_a1"), pd.DataFrame) and not globals().get("X_a1").empty:
    base = globals().get("X_a1")   # from A.1
elif isinstance(globals().get("customers_df"), pd.DataFrame) and not globals().get("customers_df").empty:
    base = globals().get("customers_df") # loaded from file


assert base is not None and not base.empty, "Run earlier C.* steps before C.n."

X_fix = base.copy()

# 2) Engineered columns we may have created earlier (only keep those that exist)
num_candidates = [
    "income_per_dependent", "log_income_per_dependent",
    "tenure_days", "tenure_years",
    "socioeconomic_score"
]
cat_candidates = ["tenure_bucket", "socioeconomic_band"]
bin_candidates = ["recent_customer"]

num_cols = [c for c in num_candidates if c in X_fix.columns]
cat_cols = [c for c in cat_candidates if c in X_fix.columns]
bin_cols = [c for c in bin_candidates if c in X_fix.columns]

# 3) Replace Inf/−Inf with NaN, then impute numerics with median
for c in num_cols:
    s = pd.to_numeric(X_fix[c], errors="coerce").replace([np.inf, -np.inf], np.nan)
    s = s.fillna(s.median() if s.notna().any() else 0)
    # sensible range constraints
    if c in {"income_per_dependent","tenure_days","tenure_years","log_income_per_dependent"}:
        s = s.clip(lower=0)
    # keep as float for safety
    X_fix[c] = s.astype(float)

# 4) Ensure binary flags are 0/1 integers
for c in bin_cols:
    s = pd.to_numeric(X_fix[c], errors="coerce").fillna(0)
    X_fix[c] = np.where(s > 0, 1, 0).astype(int)

# 5) Ensure engineered categoricals are category dtype and mode-impute
for c in cat_cols:
    col = X_fix[c].astype("string").str.strip()
    if col.isna().any():
        col = col.fillna(col.mode().iloc[0] if col.mode().size else "Unknown")
    X_fix[c] = col.astype("category")

# 6) Output for downstream steps
X_fixed_engineered = X_fix.copy()
print({
    "rows": len(X_fixed_engineered),
    "fixed_numeric": num_cols,
    "fixed_binary": bin_cols,
    "fixed_categoricals": cat_cols
})
display(X_fixed_engineered.head(3))

In [ ]:

feature_engineering_n_explanations = """
**Issue fixed:** Post-engineering artefacts — NaN/Inf values, wrong dtypes, and invalid ranges.

**Why it matters**
• Newly created features often contain {np.inf}/−{np.inf} (e.g., logs) or NaNs from division;
  these break scalers/encoders and can bias model training.

**What I did**
• Replaced Inf/−Inf with NaN and **median-imputed** engineered numerics; clipped to sensible ranges (≥0 for tenure/income features).
• Forced binary features (e.g., `recent_customer`) to **0/1 int**.
• Cast engineered buckets (e.g., `tenure_bucket`, `socioeconomic_band`) to **category** and mode-imputed.

**Impact**
• Clean, consistent engineered features with stable dtypes; ready for encoding/scaling and downstream modelling.
"""

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='feature_engineering_n_explanations', value=feature_engineering_n_explanations)

---
## D. Data Preparation for Modeling

### D.1 Split Datasets


In [ ]:
# Robust 70/15/15 split (no ambiguous truth-values; stratify only when valid)

import pandas as pd, numpy as np
from sklearn.model_selection import train_test_split

RANDOM_STATE = 42
TEST_FRAC = 0.15
VAL_FRAC  = 0.15           # overall validation share
VAL_WITHIN = VAL_FRAC / (1 - TEST_FRAC)   # ≈ 0.1765 of the train+val pool

# 1) pick the latest prepared table available
candidates = [
    "X_fixed_engineered","X_soc","X_ten","X_feat",
    "X_cleanN","X_clean3","X_clean2","X_clean1",
    "X_an","X_a2","X_a1","customers","sales_2022","sales_2021","sales_2020"
]
base = None; src_name = None
for n in candidates:
    obj = globals().get(n)
    if isinstance(obj, pd.DataFrame) and not obj.empty:
        base, src_name = obj.copy(), n
        break
if base is None:
    raise RuntimeError("No non-empty DataFrame found. Load/prepare data before splitting.")

# 2) detect target (optional)
y_col = (globals().get("y_col") or
         next((c for c in ["credit_rating","churn","target","label","segment"] if c in base.columns), None))

# exclude ID/date-like columns from features
id_like   = [c for c in ["CustomerID","customer_id","CustomerKey","customer_key","id","ID"] if c in base.columns]
date_like = [c for c in base.columns if "date" in c.lower()]
exclude   = set(id_like + date_like + ([y_col] if y_col is not None else []))

X_all = base.drop(columns=list(exclude), errors="ignore")
y_all = None
if y_col is not None:
    y_all = base[y_col]
    # if someone made target a single-column DataFrame, convert to Series
    if isinstance(y_all, pd.DataFrame) and y_all.shape[1] == 1:
        y_all = y_all.iloc[:, 0]

# helper to decide stratification only when valid
def _make_strata(y: pd.Series):
    if not isinstance(y, pd.Series):
        return None
    is_class = (y.dtype == "object") or (str(y.dtype) == "category") or (y.nunique(dropna=True) <= 20)
    if not is_class:
        return None
    vc = y.value_counts()
    # need at least 3 per class to safely split 70/15/15
    return y.astype(str) if (len(vc) > 0 and vc.min() >= 3) else None

# 3) split
if y_all is None:
    # unsupervised: split X only
    X_trv, X_te = train_test_split(X_all, test_size=TEST_FRAC, random_state=RANDOM_STATE, shuffle=True)
    X_tr,  X_va = train_test_split(X_trv, test_size=VAL_WITHIN,  random_state=RANDOM_STATE, shuffle=True)
    y_tr = y_va = y_te = None
    problem = "unsupervised"
else:
    # supervised: include y; stratify when appropriate
    strata = _make_strata(y_all)
    X_trv, X_te, y_trv, y_te = train_test_split(
        X_all, y_all, test_size=TEST_FRAC, random_state=RANDOM_STATE, shuffle=True, stratify=strata
    )
    strata_trv = _make_strata(y_trv)
    X_tr, X_va, y_tr, y_va = train_test_split(
        X_trv, y_trv, test_size=VAL_WITHIN, random_state=RANDOM_STATE, shuffle=True, stratify=strata_trv
    )
    problem = "classification" if strata is not None else "regression"

# 4) expose to globals for later cells
globals().update(dict(X_tr=X_tr, X_va=X_va, X_te=X_te, y_tr=y_tr, y_va=y_va, y_te=y_te))

print({
    "base_source": src_name,
    "problem": problem,
    "target": y_col,
    "excluded_cols": sorted(list(exclude)),
    "X_train": X_tr.shape, "X_valid": X_va.shape, "X_test": X_te.shape,
    "y_present": y_all is not None
})


In [ ]:
# <Student to fill this section>
data_splitting_explanations = """
Choosing the appropriate strategy for splitting the data will help ensure that your machine learning model generalizes well on data it has not seen, and that the model performance evaluation is as realistic as possible. Now, considering this customer dataset, common strategies include:

Random Split: This is the most straightforward approach, where data gets randomly allocated across training, validation, and testing sets. For instance, it could be 70% training, 15% validation, and 15% test. It is appropriate when data is assumed to be i.i.d. and there are no specific temporal or grouping structures to be kept by implementing a random split.

Stratified Split: If a target variable for some supervised task is introduced, a stratified split will ensure the proportion of the target variable categories is maintained in each split-that is, train, validation, and test. This is important for a classification problem with classes that are imbalanced so that splits where some classes are poorly represented or absent are avoided; hence, the cell includes logic to attempt stratification if a suitable target variable is present.

Time-based Split: In the case of a task that involves predicting future customer behavior, where the data contains a time component (e.g., date of customer acquisition, last transaction date), splitting should be based on time, such that the model is trained on data occurring up until a specified date and tested on data post-occurrence of that date. This dataset includes date information such as birth_date.

Grouped Split: When the data have natural groupings (for example, customer records associated with the same household, or sales data associated with the same sales territory), and you want to test the model's ability to generalize to completely new groups, then the split should be done such that samples from a specific group are kept within the same set, either in the train, validation, or test set. This dataset contains customer_id and territorykey, which could be used for grouping. Because the problem at hand is considered unsupervised, this simple random split is a reasonable default. If you later define a supervised task, you will want to consider whether stratification, time-based, or grouped splitting would better allow proper evaluation of your model's performance on that particular task.
"""

In [ ]:
# Do not modify this code
print_tile(size="h3", key='data_splitting_explanations', value=data_splitting_explanations)

### D.2 Data Transformation <put_name_here>


In [ ]:
# numeric(impute+scale) + categorical
# Produces X_tr_tf, X_va_tf, X_te_tf as DataFrames with feature names.

import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# 1) Inputs from D.1
assert "X_tr" in globals() and "X_va" in globals() and "X_te" in globals(), \
    "Run the D.1 splitting cell first to create X_tr/X_va/X_te."

Xtr_raw, Xva_raw, Xte_raw = X_tr.copy(), X_va.copy(), X_te.copy()

# 2) Identify column types
num_cols = [c for c in Xtr_raw.columns if pd.api.types.is_numeric_dtype(Xtr_raw[c])]
cat_cols = [c for c in Xtr_raw.columns if c not in num_cols]

# 3) Define per-type pipelines
num_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler",  StandardScaler(with_mean=True, with_std=True)),
])

cat_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe",     OneHotEncoder(handle_unknown="ignore", sparse_output=False)),
])

# 4) ColumnTransformer that applies the two pipes
preproc = ColumnTransformer(
    transformers=[
        ("num", num_pipe, num_cols),
        ("cat", cat_pipe, cat_cols),
    ],
    remainder="drop",
    verbose_feature_names_out=False
)

# 5) Fit on TRAIN ONLY, then transform all splits
preproc.fit(Xtr_raw)

def _to_df(arr, transformer, num_cols, cat_cols):
    # Build feature names from the fitted transformer
    try:
        names = transformer.get_feature_names_out()
    except Exception:
        # Fallback: compose manually if needed
        names = list(num_cols)
        if cat_cols:
            ohe = transformer.named_transformers_["cat"]["ohe"]
            cat_names = ohe.get_feature_names_out(cat_cols).tolist()
            names += cat_names
    return pd.DataFrame(arr, columns=names, index=None)

X_tr_tf = _to_df(preproc.transform(Xtr_raw), preproc, num_cols, cat_cols)
X_va_tf = _to_df(preproc.transform(Xva_raw), preproc, num_cols, cat_cols)
X_te_tf = _to_df(preproc.transform(Xte_raw), preproc, num_cols, cat_cols)

# 6) Expose objects for downstream notebooks
globals().update(dict(preproc=preproc, X_tr_tf=X_tr_tf, X_va_tf=X_va_tf, X_te_tf=X_te_tf))

print({
    "num_cols": len(num_cols),
    "cat_cols": len(cat_cols),
    "X_tr_tf": X_tr_tf.shape,
    "X_va_tf": X_va_tf.shape,
    "X_te_tf": X_te_tf.shape
})
display(X_tr_tf.head(3))

In [ ]:

data_transformation_1_explanations = """
**Transformations applied:**
• **Numeric**: median imputation → standard scaling. Median is robust to outliers; scaling centers features and
  gives comparable magnitude, helping regularised linear models and distance-based learners.
• **Categorical**: most-frequent imputation → one-hot encoding with `handle_unknown='ignore'` so unseen levels at
  validation/test don’t break the pipeline.

**Why this is appropriate here**
• The dataset mixes numeric and categorical fields (per EDA/Preparation). This pipeline preserves signal while
  preventing leakage (fit on train only) and keeps evaluation fair across splits.

**Outputs**
• `X_tr_tf`, `X_va_tf`, `X_te_tf` are pandas DataFrames with explicit feature names, ready for the Baseline/Model notebooks.
"""

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='data_transformation_1_explanations', value=data_transformation_1_explanations)

### D.3 Data Transformation <put_name_here>

In [ ]:
# Numeric: median impute -> QuantileTransformer(output='normal')

import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, QuantileTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# 1) Get raw splits from D.1
assert all(k in globals() for k in ["X_tr","X_va","X_te"]), "Run D.1 first."
Xtr_raw, Xva_raw, Xte_raw = X_tr.copy(), X_va.copy(), X_te.copy()

# 2) Column types from train
num_cols = [c for c in Xtr_raw.columns if pd.api.types.is_numeric_dtype(Xtr_raw[c])]
cat_cols = [c for c in Xtr_raw.columns if c not in num_cols]

# 3) Define per-type pipelines (Yeo-Johnson-like, rank-based Gaussianisation)
num_q_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("qtf",     QuantileTransformer(output_distribution="normal", subsample=100000, random_state=42)),
])

cat_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe",     OneHotEncoder(handle_unknown="ignore", sparse_output=False)),
])

# 4) ColumnTransformer combining both
preproc_q = ColumnTransformer(
    [("num", num_q_pipe, num_cols),
     ("cat", cat_pipe,   cat_cols)],
    remainder="drop",
    verbose_feature_names_out=False
)

# 5) Fit on TRAIN ONLY, transform all splits
preproc_q.fit(Xtr_raw)

def _to_df(arr, transformer, num_cols, cat_cols):
    try:
        names = transformer.get_feature_names_out()
    except Exception:
        # Fallback: compose OHE names manually if needed
        names = list(num_cols)
        if cat_cols:
            ohe = transformer.named_transformers_["cat"]["ohe"]
            names += ohe.get_feature_names_out(cat_cols).tolist()
    return pd.DataFrame(arr, columns=names)

X_tr_tf_q = _to_df(preproc_q.transform(Xtr_raw), preproc_q, num_cols, cat_cols)
X_va_tf_q = _to_df(preproc_q.transform(Xva_raw), preproc_q, num_cols, cat_cols)
X_te_tf_q = _to_df(preproc_q.transform(Xte_raw), preproc_q, num_cols, cat_cols)

# 6) Expose for downstream use
globals().update(dict(preproc_q=preproc_q,
                      X_tr_tf_q=X_tr_tf_q, X_va_tf_q=X_va_tf_q, X_te_tf_q=X_te_tf_q))

print({
    "num_cols": len(num_cols),
    "cat_cols": len(cat_cols),
    "X_tr_tf_q": X_tr_tf_q.shape,
    "X_va_tf_q": X_va_tf_q.shape,
    "X_te_tf_q": X_te_tf_q.shape
})
display(X_tr_tf_q.head(3))

In [ ]:

data_transformation_2_explanations = """
**What I did**
• Built an alternative preprocessing pipeline:
  – **Numeric** → median imputation then **QuantileTransformer(output='normal')** to Gaussianise skewed features.
  – **Categorical** → most-frequent imputation then One-Hot Encoding with unknowns ignored.
• Fitted the transformers **only on the training split** and applied to validation/test.

**Why**
• Many numeric variables in retail data are right-skewed (spend, frequency, amounts). Mapping them to a
  quasi-normal distribution can improve linear models and distance-based learners by reducing the influence of outliers.
• Keeping the same categorical treatment ensures comparability with D.2 while isolating the impact of the numeric transform.

**Outputs**
• `X_tr_tf_q`, `X_va_tf_q`, `X_te_tf_q` — transformed DataFrames with feature names.
"""

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='data_transformation_2_explanations', value=data_transformation_2_explanations)

### D.4 Data Transformation <put_name_here>


In [ ]:

# Numeric: median impute -> RobustScaler   (robust to outliers)
# Categorical: most-frequent impute -> group rare levels -> OneHotEncoder
# Outputs: X_tr_tf_r, X_va_tf_r, X_te_tf_r

import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# 1) Raw splits from D.1
assert all(k in globals() for k in ["X_tr","X_va","X_te"]), "Run D.1 first."
Xtr_raw, Xva_raw, Xte_raw = X_tr.copy(), X_va.copy(), X_te.copy()

# 2) Column types from train
num_cols = [c for c in Xtr_raw.columns if pd.api.types.is_numeric_dtype(Xtr_raw[c])]
cat_cols = [c for c in Xtr_raw.columns if c not in num_cols]

# 3) Small helper: group rare categories by frequency learned on TRAIN
class RareGrouper(BaseEstimator, TransformerMixin):
    def __init__(self, min_freq=0.01):  # fraction of rows; e.g., 1%
        self.min_freq = float(min_freq)
        self.keep_levels_ = {}

    def fit(self, X, y=None):
        X = pd.DataFrame(X).copy()
        for col in X.columns:
            vc = X[col].astype("string").value_counts(normalize=True, dropna=False)
            self.keep_levels_[col] = set(vc[vc >= self.min_freq].index.tolist())
        return self

    def transform(self, X):
        X = pd.DataFrame(X).copy()
        for col in X.columns:
            keep = self.keep_levels_.get(col, set())
            X[col] = X[col].astype("string").where(X[col].astype("string").isin(keep), other="Other")
        return X

# 4) Define per-type pipelines
num_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler",  RobustScaler(with_centering=True, with_scaling=True)),
])

cat_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("rare",    RareGrouper(min_freq=0.01)),                      # group train-rare levels as "Other"
    ("ohe",     OneHotEncoder(handle_unknown="ignore", sparse_output=False)),
])

# 5) ColumnTransformer combining both
preproc_r = ColumnTransformer(
    [("num", num_pipe, num_cols),
     ("cat", cat_pipe,   cat_cols)],
    remainder="drop",
    verbose_feature_names_out=False
)

# 6) Fit on TRAIN ONLY, transform all splits
preproc_r.fit(Xtr_raw)

def _to_df(arr, transformer, num_cols, cat_cols):
    try:
        names = transformer.get_feature_names_out()
    except Exception:
        names = list(num_cols)
        if cat_cols:
            ohe = transformer.named_transformers_["cat"]["ohe"]
            names += ohe.get_feature_names_out(cat_cols).tolist()
    return pd.DataFrame(arr, columns=names)

X_tr_tf_r = _to_df(preproc_r.transform(Xtr_raw), preproc_r, num_cols, cat_cols)
X_va_tf_r = _to_df(preproc_r.transform(Xva_raw), preproc_r, num_cols, cat_cols)
X_te_tf_r = _to_df(preproc_r.transform(Xte_raw), preproc_r, num_cols, cat_cols)

# 7) Expose for downstream notebooks
globals().update(dict(preproc_r=preproc_r,
                      X_tr_tf_r=X_tr_tf_r, X_va_tf_r=X_va_tf_r, X_te_tf_r=X_te_tf_r))

print({
    "num_cols": len(num_cols),
    "cat_cols": len(cat_cols),
    "X_tr_tf_r": X_tr_tf_r.shape,
    "X_va_tf_r": X_va_tf_r.shape,
    "X_te_tf_r": X_te_tf_r.shape
})
display(X_tr_tf_r.head(3))

In [ ]:

data_transformation_3_explanations = """
**What I did**
• **Numeric** → median imputation then **RobustScaler**, which scales by the IQR and is less sensitive to outliers
  than StandardScaler.
• **Categorical** → most-frequent imputation, then a **rare-category grouper** that merges very small levels (<1% of
  train) into 'Other' before One-Hot Encoding (with unknowns ignored).

**Why**
• Retail-style features often contain extreme values; RobustScaler stabilises linear and distance-based models.
• Grouping rare categories prevents very wide, sparse matrices and reduces variance in coefficients while keeping
  interpretability (you still see a clean 'Other' bucket).

**Outputs**
• `X_tr_tf_r`, `X_va_tf_r`, `X_te_tf_r` — transformed DataFrames with feature names. Use alongside D.2 and D.3 to
  pick the best preprocessing via validation performance.
"""

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='data_transformation_3_explanations', value=data_transformation_3_explanations)

### D.n Fixing "\<describe_issue_here\>"

> You can add more cells related to data preparation in this section

In [ ]:
# generic data-prep fixes: drop constants/dupes, clip extreme outliers, align columns
import pandas as pd
import numpy as np

# 1) Pick inputs: prefer transformed sets (D.4 → D.3 → D.2), then raw (D.1)
def _first_df(*names):
    for n in names:
        obj = globals().get(n)
        if isinstance(obj, pd.DataFrame) and not obj.empty:
            return obj.copy(), n
    return None, None

Xtr, src_tr = _first_df("X_tr_tf_r", "X_tr_tf_q", "X_tr_tf", "X_tr")
Xva, src_va = _first_df("X_va_tf_r", "X_va_tf_q", "X_va_tf", "X_va")
Xte, src_te = _first_df("X_te_tf_r", "X_te_tf_q", "X_te_tf", "X_te")

assert Xtr is not None and Xva is not None and Xte is not None, "Run D.1 and at least one of D.2–D.4."

# 2) Drop constant columns (zero variance) on TRAIN, then apply same mask to VA/TEST
const_cols = Xtr.columns[Xtr.nunique(dropna=False) <= 1].tolist()
if const_cols:
    Xtr = Xtr.drop(columns=const_cols)
    Xva = Xva.drop(columns=[c for c in const_cols if c in Xva], errors="ignore")
    Xte = Xte.drop(columns=[c for c in const_cols if c in Xte], errors="ignore")

# 3) Drop duplicate columns w.r.t. TRAIN (exact duplicates)
def _drop_dup_cols(df):
    # transpose & drop_duplicates on rows -> back to columns
    mask = ~df.T.duplicated()
    return df.loc[:, mask], df.columns[~mask].tolist()

Xtr, dup_cols = _drop_dup_cols(Xtr)
if dup_cols:
    Xva = Xva.drop(columns=[c for c in dup_cols if c in Xva], errors="ignore")
    Xte = Xte.drop(columns=[c for c in dup_cols if c in Xte], errors="ignore")

# 4) Robust outlier clipping on TRAIN numeric columns (winsorize to 1st–99th pct)
num_cols = [c for c in Xtr.columns if pd.api.types.is_numeric_dtype(Xtr[c])]
if num_cols:
    q_low  = Xtr[num_cols].quantile(0.01)
    q_high = Xtr[num_cols].quantile(0.99)
    Xtr[num_cols] = Xtr[num_cols].clip(lower=q_low, upper=q_high, axis=1)
    # Use the same cut points learned on TRAIN for VA/TEST
    Xva[num_cols] = Xva[num_cols].clip(lower=q_low, upper=q_high, axis=1)
    Xte[num_cols] = Xte[num_cols].clip(lower=q_low, upper=q_high, axis=1)

# 5) Align columns across splits (in case upstream pipelines produced slight mismatches)
common_cols = sorted(list(set(Xtr.columns) & set(Xva.columns) & set(Xte.columns)))
Xtr_fix = Xtr[common_cols].copy()
Xva_fix = Xva[common_cols].copy()
Xte_fix = Xte[common_cols].copy()

# 6) Expose results and a small report
globals().update(dict(X_tr_fix=Xtr_fix, X_va_fix=Xva_fix, X_te_fix=Xte_fix))

report = {
    "source_train": src_tr, "source_valid": src_va, "source_test": src_te,
    "dropped_constants": const_cols,
    "dropped_duplicates": dup_cols,
    "num_cols_clipped": len(num_cols),
    "final_shapes": {"train": Xtr_fix.shape, "valid": Xva_fix.shape, "test": Xte_fix.shape},
}
print(report)
display(Xtr_fix.head(3))

In [ ]:

data_transformation_n_explanations = """
**Issue fixed:** Residual prep artefacts after earlier steps — constant/duplicate features, extreme values,
and column mismatches across splits.

**What I did**
• Dropped **zero-variance** and **duplicate** columns (learned on train, applied to val/test).
• Applied **robust winsorisation** (clip to 1st–99th percentiles) on numeric features using train cut-points to reduce
  the influence of extreme outliers without discarding data.
• **Aligned columns** to ensure train/validation/test have identical feature sets.

**Why it matters**
• Constant/duplicate columns add noise and can inflate model complexity without adding signal.
• Extreme values can dominate loss/gradients and harm generalisation.
• Misaligned columns silently break training/evaluation. These fixes make the dataset stable and reproducible.

**Outputs**
• Cleaned matrices: `X_tr_fix`, `X_va_fix`, `X_te_fix` with the same columns and shapes: {{
    "train": {globals().get('X_tr_fix').shape if 'X_tr_fix' in globals() else 'n/a'},
    "valid": {globals().get('X_va_fix').shape if 'X_va_fix' in globals() else 'n/a'},
    "test":  {globals().get('X_te_fix').shape if 'X_te_fix' in globals() else 'n/a'}
}}.
"""

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL
print_tile(size="h3", key='data_transformation_n_explanations', value=data_transformation_n_explanations)

In [ ]:
# Bridge variables for the Save Datasets cell

import pandas as pd
import numpy as np

def _pick_df(*candidates):
    """Return the first non-empty DataFrame among the given global names."""
    for name in candidates:
        obj = globals().get(name)
        if isinstance(obj, pd.DataFrame) and not obj.empty:
            return obj.copy()
    return None

def _ensure_df(x):
    if x is None:
        return None
    if isinstance(x, pd.DataFrame):
        return x.copy()
    # numpy array -> DataFrame
    try:
        return pd.DataFrame(x).copy()
    except Exception:
        return None

def _pick_y(*names):
    """Pick a y Series if available; else return None (unsupervised)."""
    for n in names:
        obj = globals().get(n)
        if obj is not None:
            # DataFrame with one column -> Series
            if isinstance(obj, pd.DataFrame) and obj.shape[1] == 1:
                return obj.iloc[:, 0].copy()
            # Series ok
            if isinstance(obj, (pd.Series, pd.Index)):
                return pd.Series(obj).copy()
    return None

# Prefer the most prepared matrices; fall back to raw
X_train = _ensure_df(_pick_df("X_tr_fix", "X_tr_tf_r", "X_tr_tf_q", "X_tr_tf", "X_tr"))
X_val   = _ensure_df(_pick_df("X_va_fix", "X_va_tf_r", "X_va_tf_q", "X_va_tf", "X_va"))
X_test  = _ensure_df(_pick_df("X_te_fix", "X_te_tf_r", "X_te_tf_q", "X_te_tf", "X_te"))

# Targets (may be None for unsupervised datasets)
y_train = _pick_y("y_tr")
y_val   = _pick_y("y_va")
y_test  = _pick_y("y_te")

# If no targets exist (unsupervised), create empty Series so the save cell still runs
target_name = globals().get("y_col", "target")
if y_train is None:
    y_train = pd.Series([np.nan]*len(X_train), name=target_name)
if y_val is None:
    y_val = pd.Series([np.nan]*len(X_val), name=target_name)
if y_test is None:
    y_test = pd.Series([np.nan]*len(X_test), name=target_name)

# Final sanity prints (optional)
print("Ready to save:",
      {"X_train": X_train.shape, "X_val": X_val.shape, "X_test": X_test.shape,
       "y_train_len": len(y_train), "y_val_len": len(y_val), "y_test_len": len(y_test)})

---
## E. Save Datasets

> Do not change this code

In [ ]:
# DO NOT MODIFY THE CODE IN THIS CELL

try:
  X_train.to_csv(at.folder_path / 'X_train.csv', index=False)
  y_train.to_csv(at.folder_path / 'y_train.csv', index=False)

  X_val.to_csv(at.folder_path / 'X_val.csv', index=False)
  y_val.to_csv(at.folder_path / 'y_val.csv', index=False)

  X_test.to_csv(at.folder_path / 'X_test.csv', index=False)
  y_test.to_csv(at.folder_path / 'y_test.csv', index=False)
except Exception as e:
  print(e)